<a href="https://colab.research.google.com/github/anumit-web/ML-Analytics-Portfolio-2024/blob/main/2.%20Sales%20Forecasting/Sales_Forecasting_time_series_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

time series

sales forecast

In [15]:
print('Hello')

Hello


In [16]:
from datetime import datetime
print(datetime.now())

2024-09-10 21:06:37.695313


In [17]:
import pandas as pd

In [18]:
data_df = pd.read_csv('train.csv')

In [19]:
data_df.head()

,Date,store,product,number_sold
0,2010-01-01,0.0,0.0,801.0
1,2010-01-02,0.0,0.0,810.0
2,2010-01-03,0.0,0.0,818.0
3,2010-01-04,0.0,0.0,796.0
4,2010-01-05,0.0,0.0,808.0


In [20]:
data_df.tail()

,Date,store,product,number_sold
164885,2011-06-20,5.0,0.0,787.0
164886,2011-06-21,5.0,0.0,809.0
164887,2011-06-22,5.0,0.0,817.0
164888,2011-06-23,5.0,0.0,802.0
164889,2011-06-24,NaN,NaN,NaN


In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164890 entries, 0 to 164889
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Date         164890 non-null  object 
 1   store        164889 non-null  float64
 2   product      164889 non-null  float64
 3   number_sold  164889 non-null  float64
dtypes: float64(3), object(1)
memory usage: 5.0+ MB


In [22]:
data_df.describe()

,store,product,number_sold
count,164889.000000,164889.000000,164889.000000
mean,2.009807,4.485290,740.748795
std,1.422251,2.879073,224.772633
min,0.000000,0.000000,238.000000
25%,1.000000,2.000000,671.000000
50%,2.000000,4.000000,785.000000
75%,3.000000,7.000000,904.000000
max,5.000000,9.000000,1205.000000


In [23]:
data_df.isnull().sum()

,0
Date,0
store,1
product,1
number_sold,1


In [24]:
data_df.agg(['min','max'])

,Date,store,product,number_sold
min,2010-01-01,0.0,0.0,238.0
max,2018-12-31,5.0,9.0,1205.0
